In [1]:
import pandas as pd
import tensorflow as tf

from utils.all import *

print(tf.VERSION)

1.6.0


In [2]:
#加载数据集
x_train, y_train, x_test, y_test = load_dataset1('d:/abc11.csv')
#特征向量维度
m_samples, n_features = x_train.shape
print(n_features)

(82731, 46)
45


In [3]:
#网络结构
_x = tf.placeholder(tf.float32, shape=(None, n_features), name='input')
_y = tf.placeholder(tf.float32, shape=(None, 1))

L1 = tf.layers.Dense(20, activation=tf.nn.relu)(_x)
L2 = tf.layers.Dense(10, activation=tf.nn.relu)(L1)
L3 = tf.layers.Dense(1)(L2)
logits = tf.nn.sigmoid(L3, name='output')
y_predict = tf.where(logits < 0.5, tf.zeros_like(logits), tf.ones_like(logits), name='y_predict')

In [4]:
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=_y, logits=L3))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_op)

In [5]:
precision_op = 1 - tf.reduce_mean(tf.abs(y_predict - _y))
c = _y * y_predict
accuracy_op = tf.count_nonzero(c) / tf.count_nonzero(y_predict)
recall_op = tf.count_nonzero(c) / tf.count_nonzero(_y)

In [6]:
init_op = tf.global_variables_initializer()

In [7]:
with tf.Session() as session:
    session.run(init_op)
    for i in range(1500):
        session.run(train_op, feed_dict={_x: x_train, _y: y_train})
        loss, precision = session.run([loss_op, precision_op], feed_dict={_x: x_train, _y: y_train})
        loss_test, precision_test = session.run([loss_op, precision_op], feed_dict={_x: x_test, _y: y_test})
        if i % 100 == 0:
            print('epoch: ', '%04d' % (i + 1), 'loss: ', "{:.4f}".format(loss), 'precision: ', "{:.4f}".format(precision),
                  'loss_test: ', "{:.4f}".format(loss_test), 'precision_test: ', "{:.4f}".format(precision_test))
    print('acc and recall：', session.run([accuracy_op, recall_op], feed_dict={_x: x_train, _y: y_train}))
    print('acc_test and recall_test：', session.run([accuracy_op, recall_op], feed_dict={_x: x_test, _y: y_test}))
    #保存模型
#     builder = tf.saved_model.builder.SavedModelBuilder('d:/tmp/m_full7')
#     builder.add_meta_graph_and_variables(
#         session,
#         [tf.saved_model.tag_constants.SERVING]
#     )
#     builder.save()
    save_model(session, 'd:/tmp/m_full89')

epoch:  0001 loss:  0.8968 precision:  0.1602 loss_test:  0.8995 precision_test:  0.1597
epoch:  0101 loss:  0.1181 precision:  0.9703 loss_test:  0.1161 precision_test:  0.9708
epoch:  0201 loss:  0.0415 precision:  0.9943 loss_test:  0.0390 precision_test:  0.9944
epoch:  0301 loss:  0.0242 precision:  0.9965 loss_test:  0.0219 precision_test:  0.9969
epoch:  0401 loss:  0.0166 precision:  0.9974 loss_test:  0.0146 precision_test:  0.9976
epoch:  0501 loss:  0.0128 precision:  0.9978 loss_test:  0.0112 precision_test:  0.9977
epoch:  0601 loss:  0.0107 precision:  0.9979 loss_test:  0.0094 precision_test:  0.9979
epoch:  0701 loss:  0.0093 precision:  0.9980 loss_test:  0.0085 precision_test:  0.9979
epoch:  0801 loss:  0.0084 precision:  0.9981 loss_test:  0.0079 precision_test:  0.9981
epoch:  0901 loss:  0.0077 precision:  0.9983 loss_test:  0.0075 precision_test:  0.9983
epoch:  1001 loss:  0.0071 precision:  0.9983 loss_test:  0.0071 precision_test:  0.9983
epoch:  1101 loss:  0

In [ ]:
np.count_nonzero(y_train)

In [ ]:
np.count_nonzero(y_test)

In [ ]:
print('数据集大小：', np.size(y_train)+np.size(y_test))
print('训练集大小：', np.size(y_train), '，非0数：', np.count_nonzero(y_train))
print('测试集大小：', np.size(y_test), '，非0数：', np.count_nonzero(y_test))